## API EDA PROCESS

Luis Angel Garcia (2230177)

Libraries

In [17]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

import sys

sys.path.append("../config_documents")

import postgres_setup as pg
import importlib
importlib.reload(pg)

<module 'postgres_setup' from '/mnt/d/Universidad/ETL/workshop2_spotify/notebook/../config_documents/postgres_setup.py'>

External configuration load

In [8]:
def leer_configuracion_spotify(ruta_archivo):
    config = {}
    with open(ruta_archivo, 'r') as archivo:
        for linea in archivo:
            if '=' in linea:
                clave, valor = linea.strip().split('=', 1)
                config[clave.strip()] = valor.strip()
    return config

config = leer_configuracion_spotify("../config_documents/spotify_config.txt")

read artist from spotify_api_artist

In [12]:
artistas_colombianos = []
with open("../config_documents/spotify_api_artist.txt", 'r', encoding='utf-8') as archivo:
    for linea in archivo:
        if linea.strip():
            artistas_colombianos.append(linea.strip())

User autentification

In [13]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=config['CLIENT_ID'],
    client_secret=config['CLIENT_SECRET']
))

search artist from api

In [14]:
artistas_data = []
for nombre in artistas_colombianos:
    resultado = sp.search(q=f"artist:{nombre}", type='artist', limit=1)
    if resultado['artists']['items']:
        artista = resultado['artists']['items'][0]
        artistas_data.append({
            'artist_id': artista['id'],
            'artist_name': artista['name'],
            'popularity': artista['popularity'],
            'followers': artista['followers']['total'],
            'genres': ', '.join(artista['genres'])
        })

DF with user TOP artist

In [ ]:
df = pd.DataFrame(artistas_data)
df

load to postgres

In [18]:
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
conn = pg.conectar_etl()
pg.crear_tabla_spotify_artists_api(conn)
pg.insertar_datos_spotify_artists_api(conn,df)

Tabla 'spotify_artists_api' verificada/creada.
Datos insertados en 'spotify_artists_api'
